In [93]:
import ee
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as PILImage
import requests
from io import BytesIO
from IPython.display import Image
ee.Authenticate()

ee.Initialize(project='ee-philipkenneweg')




# Define the urban location of interest as a point near Lyon, France.
u_lon = 8.5325
u_lat = 52.03202
u_poi = ee.Geometry.Point(u_lon, u_lat)
roi = u_poi.buffer(3e3)

def get_img_from_url(url):
    response = requests.get(url)
    img = PILImage.open(BytesIO(response.content))

    # Convert the image to a NumPy array
    img_array = np.array(img)

    # print(img_array.shape)
    # print(img_array.dtype)
    # print(img_array.min(), img_array.max())

    # plt.imshow(img_array)
    # plt.show()
    return img_array

def get_temp_img(i_date, f_date):

    # Import the MODIS land surface temperature collection.
    lst = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')#ee.ImageCollection('MODIS/006/MOD11A1')
    # Selection of appropriate bands and dates for LST.
    lst = lst.select('temperature_2m').filterDate(i_date, f_date)
   # print(lst.size().getInfo())
    if lst.size().getInfo() == 0:
        print('No temp data available for the given time period.')
        return
    # Reduce the LST collection by mean.
    lst_img = lst.mean()
    # Convert Kelvin to Celsius.
    lst_img = lst_img.select('temperature_2m').add(-273.15)

    url = lst_img.getThumbUrl({
        'min': -20, 'max':  30,'dimensions': 512, 'region': roi})#,'palette': ['blue', 'yellow', 'orange', 'red']})

    array = get_img_from_url(url)
    
    array = array*(50/255)-20.0
    print(array.min(), array.max())
    np.save('data/'+i_date+'temp.npy', array)

    return array

def get_tmp_graph(i_date, f_date):

    # Import the MODIS land surface temperature collection.
    lst = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')#ee.ImageCollection('MODIS/006/MOD11A1')
    # Selection of appropriate bands and dates for LST.
    lst = lst.select('temperature_2m').filterDate(i_date, f_date)
   # print(lst.size().getInfo())
    if lst.size().getInfo() == 0:
        print('No temp data available for the given time period.')
        return
    # Reduce the LST collection by mean.
    #for 
    lst_u_poi = lst.getRegion(u_poi, 1000).getInfo()
    tmp_list = []
    for i in range(1,len(lst_u_poi)):
      # print(lst_u_poi[i][4])
        tmp_list.append(lst_u_poi[i][4])
   # print(lst_u_poi)
    lst_u_poi  = np.asarray(tmp_list)-273.15
   # for i in range(1,128):
    return lst_u_poi
   # print(lst_u_poi)
   # print(lst_u_poi[:5])
   # print(lst.mean().sample(u_poi).first().get('temperature_2m').getInfo())
   


def get_satellite_img(i_date, f_date):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(roi)
    .filterDate(i_date, f_date)
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)))

    #print(s2_sr_col.size().getInfo())
    if s2_sr_col.size().getInfo() == 0:
        print('No img data available for the given time period.')
        return
    sl2_img = s2_sr_col.mean()
    url = sl2_img.getThumbURL({'dimensions': 512, 'region': roi,'bands': ['B4', 'B3', 'B2'], "min": 0, "max": 3000})

    array = get_img_from_url(url)
    np.save('data/'+i_date+'rgb.npy', array)
    return array

def get_landcover_img(i_date, f_date):
    col = ee.ImageCollection("COPERNICUS/CORINE/V20/100m").filterBounds(roi).filterDate(i_date, f_date)

    if s2_sr_col.size().getInfo() == 0:
        print('No img data available for the given time period.')
        return

    img = col.mean()
    url = img.getThumbURL({'dimensions': 512, 'region': roi,'bands': ['landcover']})
    array = get_img_from_url(url)
    np.save('data/'+i_date+'landcover.npy', array)
    return array


for i in range(12):
    print("Current month", i+1)
    i_date = "2024-"+str(i+1).zfill(2)+"-01"
    f_date = "2024-"+str(i+1).zfill(2)+"-28"
    list_tmps = get_tmp_graph(i_date, f_date)
    img = get_satellite_img(i_date, f_date)
    if img is None:
        continue
   # plt.imshow(img)
    # temp = get_temp_img(i_date, f_date)
    # if temp is None:
    #     continue
    # cover = get_landcover_img(i_date, f_date)
    # if cover is None:
    #     continue

   # print(cover)
    fix, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img)
    ax[1].plot(list_tmps)
    #ax[1].imshow(temp)
   #ax[2].imshow(cover)
    plt.show()





Current month 1


NameError: name 'list_u_poi' is not defined

In [69]:
s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(roi)
    .filterDate('2020-01-01', '2020-06-30')
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)))

roi = u_poi.buffer(3e3)
print(s2_sr_col.size().getInfo())
sl2_img = s2_sr_col.mean()
url = sl2_img.getThumbURL({'dimensions': 512, 'region': roi,'bands': ['B4', 'B3', 'B2'], "min": 0, "max": 3000})

Image(url=url)


22
